In [1]:
!python --version

Python 3.6.9


In [2]:
# installing requirements
!pip install beautifulsoup4
!pip install bs4
!pip install requests
!pip install requests-cache
!pip install pandas

In [0]:
#import's dependencies
import pandas as pd
import requests
import re
import requests_cache
import sys
from bs4 import BeautifulSoup

In [0]:
# install cache
requests_cache.install_cache('dataCache')
# set recursion limite
sys.setrecursionlimit(1000000)

In [0]:
# get uri, adjusting uri
def get_uri(msg):
  return add_https(input(msg))

def add_https(uri):
  if(uri[0:4]!="http"):
    uri = "https://"+uri
  return uri

In [0]:
# settings 
fetched_links = set()
restricted_links = set()
good_codes = set() 
good_codes.update(list(range(100, 399, 1)))
keyword = ''
object_word = {
    'text':[], 
    'link': [],
    'score':[],
}

def clear():
  fetched_links.clear()
  restricted_links.clear() 
  globals()['object_word']['link'].clear()
  globals()['object_word']['score'].clear()
  globals()['object_word']['text'].clear() 



In [0]:
# handle links
def handle_links(page, depth, uri):
  for a in page.find_all('a')[0:30:1]:
    link = a.get('href')
    if link is None:
        restricted_links.add(link)
    else:
      if link[0:4] == 'http':
        search(link, keyword, depth-1)
      else:
        restricted_links.add(link)

# handle text
def handle_text(page):
    regex_words = re.compile('[a-zA-ZÀ-ú]+')
    regex_split = re.compile('\s+')

    line_list = regex_split.split(page.get_text())
       
    words = {}

    for line in line_list:
      wordRE = regex_words.match(line)
      if wordRE is not None:
        word = wordRE.group().lower()
        if word in words.keys():
          words[word] += 1;
        else:
          words[word] = 1;
    return words

# get phrase
def get_phrase(beautiful_page):
  wordkey =  globals()['keyword']
  caractere = ('.'*20)+ wordkey + ('.'*20)
  phrase = re.findall(caractere, beautiful_page.get_text(), re.IGNORECASE) 
  return phrase[0] if phrase else wordkey 

In [0]:
# get texts and adjusting links and tag's 
def handle_page(page, depth, uri):
  beautiful_page = BeautifulSoup(page.text, "html5lib")
  # handle links
  handle_links(beautiful_page, depth, uri)
  # handle text
  words = handle_text(beautiful_page)
  percent = percent_calculator(words, keyword)
  phrase = get_phrase(beautiful_page)
  # adding uri in fetched 'list'
  globals()['object_word']['link'].append(uri)
  globals()['object_word']['score'].append(percent)
  globals()['object_word']['text'].append(phrase) 
  fetched_links.add(uri)

def percent_calculator(words, keyword):    
    if keyword not in words.keys():
      return 0
    else:
      number = 0
      for key in words.keys():
        number += words[key]
      
      return words[keyword]/ number


In [0]:
# search function

def search(uri, keyword, depth=0):
  if depth >= 0: 
    # verifica se o link não está na 'lista' de restritos
    if uri not in restricted_links:
      # verifica se o link não está na 'lista' de visitados
      if uri not in fetched_links:
        response = requests.get(uri, allow_redirects=False)
        status_code = response.status_code
        print("[NEW VISIT]:",uri,"-",status_code)
        if status_code in good_codes:
          #handle http
          handle_page(response, depth, uri)
        else:
          restricted_links.add(uri)
      else:
        print("[URI HAS BEEN VISITED]:",uri)
        return
    else:
      print("[URI HAS BEEN VISITED - SOMETHING UNEXPECTED WAS DETECTED]:", uri)
      return 


In [0]:
# create dt
def dt_create(object_word):
  # print(object_word)
  return pd.DataFrame(object_word)


In [0]:
def list_links():
  # print('----- fetched links --------')
  df = dt_create(globals()['object_word'])
  return df  


In [0]:
# xiaomi  www.mi.com/br/
# python www.python.org
# coronavírus https://g1.globo.com/bemestar/coronavirus/noticia/2020/03/12/ultimas-noticias-de-coronavirus-de-12-de-marco.ghtml
# pão https://pt.wikipedia.org/wiki/P%C3%A3o
#main
def main():
  clear()
  global keyword
  keyword = input("Digite a palavra buscada: ")
  uri = get_uri("digite o link de busca inicial: ")
  depth = int(input("profundidade da busca: "))
  try:
    search(uri, keyword, depth)
    global df
  except NameError:
    print(NameError)
    df = list_links()
  df = list_links()


if __name__ == '__main__':
  main()

Digite a palavra buscada: python
digite o link de busca inicial: www.python.org
profundidade da busca: 3
[NEW VISIT]: https://www.python.org - 200
[NEW VISIT]: https://docs.python.org - 302
[NEW VISIT]: https://pypi.python.org/ - 301
[NEW VISIT]: https://www.facebook.com/pythonlang?fref=ts - 200
[NEW VISIT]: https://www.facebook.com/ - 200
[NEW VISIT]: https://www.facebook.com/ - 200
[NEW VISIT]: https://www.facebook.com/recover/initiate?lwv=110&ars=royal_blue_bar - 302
[NEW VISIT]: https://es-la.facebook.com/ - 200
[NEW VISIT]: https://fr-fr.facebook.com/ - 200
[NEW VISIT]: https://zh-cn.facebook.com/ - 200
[NEW VISIT]: https://ar-ar.facebook.com/ - 200
[NEW VISIT]: https://pt-br.facebook.com/ - 200
[NEW VISIT]: https://it-it.facebook.com/ - 200
[NEW VISIT]: https://ko-kr.facebook.com/ - 200
[NEW VISIT]: https://de-de.facebook.com/ - 200
[NEW VISIT]: https://hi-in.facebook.com/ - 200
[NEW VISIT]: https://ja-jp.facebook.com/ - 200
[NEW VISIT]: https://messenger.com/ - 301
[NEW VISIT]: 

/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://brochure.getpython.info/media/releases/psf-python-brochure-vol.-i-final-download.pdf/view" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


[NEW VISIT]: https://brochure.getpython.info/media/releases/prerelases/psf-python-brochure-vol-1-final-content-preview - 200


In [0]:
df.sort_values(by='score', ascending=False).iloc[:10]

In [0]:
df.sort_values(by='score')